# TRPO

```{note}
TRPO updates policies by taking the largest step possible to improve performance, while satisfying a special constraint on how close the new and old policies are allowed to be.
```

## Preliminaries

Consider an infinite-horizon discounted Markov decision
process (MDP), defined by the tuple $(\mathcal{S}, \mathcal{A}, P, r, \rho_{0},\gamma)$ where $\mathcal{S}$ is a finite set of states, $\mathcal{A}$ is a finite set of actions, $P: \mathcal{S}\times\mathcal{A}\times\mathcal{A}\to\mathbb{R}$ is the transition probability distriarbution, $r:\mathcal{S}\to\mathbb{R}$ is the reward function (difference from our previous setting), $\rho_{0}:\mathcal{S}\to\mathbb{R}$ is
the distribution of the initial state $s_{0}$, $\gamma\in(0, 1)$ is the
discount factor.

Let $\pi$ denote a stochastic policy $\pi:\mathcal{S}\times\mathcal{A}\to[0, 1]$, and let $\eta(\pi)$ denote its expected discounted reward:

$$\eta(\pi) = \mathbb{E}_{s_0,a_0,\dots}\left[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t})\right]$$

where $s_0\sim\rho_0(s_0)$, $a_t\sim\pi(a_t|s_t)$, $s_{t+1}\sim P(s_{t+1}|s_{t}, a_{t})$. Let $A_{\pi}(s, a) = Q_{\pi}(s, a) - V_{\pi}(s)$

The following useful identity expresses the expected return
of another policy $\tilde{\pi}$ in terms of the advantage over $\pi$:

$$\eta(\tilde{\pi}) = \eta(\pi) + \mathbb{E}_{s_0,a_0,\dots\sim\tilde{\pi}}\left[\sum_{t=0}^{\infty}\gamma^{t}A_{\pi}(s_t,a_t)\right]$$

Let $\rho_{\pi}$ be the discounted visitation frequencies:

$$\rho_{\pi}(s) = P(s_0=s) + \gamma P(s_1=s) + \gamma^{2}P(s_2=s) + \dots$$

We can rewrite Equation with a sum over states instead
of timesteps:

$$
\begin{aligned}
\eta(\tilde{\pi}) &= \eta(\pi) + \sum_{t=0}^{\infty}\sum_{s}P(s_{t}=s|\tilde{\pi})\sum_{a}\tilde{\pi}(a|s)\gamma^{t}A_{\pi}(s, a) \\
&= \eta(\pi) + \sum_{s}\sum_{t=0}^{\infty}\gamma^{t}P(s_{t}=s|\tilde{\pi})\sum_{a}\tilde{\pi}(a|s)A_{\pi}(s, a) \\
&= \eta(\pi) + \sum_{s}\rho_{\tilde{\pi}}(s)\sum_{a}\tilde{\pi}(a|s)A_{\pi}(s, a)
\end{aligned}
$$

This equation implies that any policy update $\pi\to\tilde{\pi}$ that
has a nonnegative expected advantage at every state $s$ is guaranteed to increase
the policy performance $\rho$. However, the complex dependency of $rho_{\tilde{\pi}}(s)$ on $\tilde{\pi}$ makes it difficult to optimize directly.
Instead, we introduce the following local approximation
to $\eta$:

$$L_{\pi}(\tilde{\pi}) = \eta{\pi} + \sum_{s}\rho_{\pi}(s)\sum_{a}\tilde{\pi}(a|s)A_{\pi}(s, a)$$

Note that $L_{\pi}$ uses the visitation frequency $\rho_{\pi}$ rather than $\rho_{\tilde{\pi}}$. If we have a parameterized policy $\pi_{\theta}$, where $\pi_{\theta}(a|s)$ is a differentiable function
of the parameter vector $\theta$, then $L_{\pi}$ matches $\eta$ to first order:

$$L_{\pi_{\theta_{0}}}(\pi_{\theta_{0}}) = \eta(\pi_{\theta_{0}})$$

$$\nabla_{\theta}L_{\pi_{\theta_0}}(\pi_{\theta_0})|_{\theta=\theta_0} = \nabla_{\theta}\eta(\pi_{\theta_{0}})|_{\theta=\theta_0}$$

It implies that a sufficiently small step that improves $L_{\pi_{\theta_{\text{old}}}}$ will also improve $\eta$, but does not give
us any guidance on how big of a step to take.

## Monotonic Improvement Guarantee

To address the above issue, the author proposed
a policy updating scheme, for which they could provide explicit lower bounds
on the improvement of $\eta$. Let $\pi_{\text{old}}$ denote the current policy, denote total variation divergence $D_{\text{TV}}(p\|q) = \frac{1}{2}\sum_{i}|p_{i} - q_{i}|$, define a distance measure between $\pi$ and $\tilde{\pi}$:

$$
D_{\text{TV}}^{\max}(\pi, \tilde{\pi}) = \max_{s}D_{\text{TV}}(\pi(\cdot|s)\left|\right|\tilde{\pi}(\cdot|s))
$$

**Theorem 1.** Let $\alpha=D_{\text{TV}}^{\max}(\pi_{\text{old}}, \pi_{\text{new}})$. Then the following
bound holds:

$$\eta(\pi_{\text{new}})\ge L_{\pi_{\text{old}}}(\pi_{\text{new}}) - \frac{4\epsilon\gamma}{(1-\gamma)^2}\alpha^{2}$$

where $\epsilon=\max_{s,a}|A_{\pi_{\text{old}}}(s, a)|$.

Next, we note the following relationship between the total
variation divergence and the KL divergence $D_{\text{TV}}(p\|q)^{2}\le D_{\text{KL}}(p\|q)$. Let $D_{\text{KL}}^{\max}(\pi,\tilde{\pi}) = \max_{s}D_{\text{KL}}(\pi(\cdot|s)\|\tilde{\pi}(\cdot|s))$, the following bounds the follows directly:

$$\eta(\tilde{\pi})\ge L_{\pi}(\tilde{\pi}) - CD_{\text{KL}}^{\max}(\pi,\tilde{\pi}),$$

$$\text{  where  }\  C=\frac{4\epsilon\gamma}{(1-\gamma)^2}.$$

**Algorithm 1** describes an approximate policy iteration
scheme based on the policy improvement bound above. Note that for now, we assume exact evaluation of
the advantage values $A_{\pi}$.

------------

Initialize $\pi_{0}$<br>
**for** $i=0,1,2,\dots$ until convergence **do**<br>
$\quad$Compute all advantage values $A_{\pi_{i}}(s, a)$.<br>
$\quad$Solve the constrained optimization problem<br>
$\quad \pi_{i+1} = \arg\max_{\pi}\left[L_{\pi_{i}}(\pi) - CD_{\text{KL}}^{\max}(\pi_{i}, \pi)\right]$<br>
$\quad$$\quad$where $C = 4\epsilon\gamma/(1-\gamma)^{2}$<br>
$\quad$$\quad$and $L_{\pi_{i}}(\pi) = \eta(\pi_{i}) + \sum_{s}\rho_{\pi_{i}}(s)\sum_{a}\pi(a|s)A_{\pi_{i}}(s,a)$<br>
**end for**

------------

This algorithm is a type of minorization-maximization (MM) algorithm, which is guaranteed
to generate a monotonically improving sequence of policies. Trust region policy optimization, which we propose in the
following section, is an approximation to Algorithm 1.

## Optimization of Parameterized Policies

The preceding section showed that $\eta(\theta) \ge L_{\theta_{\text{old}}}(\theta) - CD_{\text{KL}}^{\max}(\theta_{\text{old}}, \theta)$, with equality at $\theta_{\text{old}}=\theta$. Thus, by performing
the following maximization, we are guaranteed to
improve the true objective $\eta$:

$$\underset{\theta}{\text{maximize}}\left[L_{\theta_{\text{old}}}(\theta) - CD_{\text{KL}}^{\max}(\theta_{\text{old}}, \theta)\right].$$

In practice, if we used the penalty coefficient C recommended
by the theory above, the step sizes would be very
small. One way to take larger steps in a robust way is to use
a constraint on the KL divergence between the new policy
and the old policy, i.e., a trust region constraint:

$$\underset{\theta}{\text{maximize}}L_{\theta_{\text{old}}}(\theta)$$

$$\text{subject to  }D_{\text{KL}}^{\max}(\theta_{\text{old}}, \theta)\le\delta.$$

This problem imposes a constraint that the KL divergence
is bounded at every point in the state space. While it is
motivated by the theory, this problem is impractical to solve
due to the large number of constraints. Instead, we can use
a heuristic approximation which considers the average KL
divergence:

$$\bar{D}_{\text{KL}}^{\rho}(\theta_1, \theta_2) :=\mathbb{E}_{s\sim\rho}\left[D_{\text{KL}}(\pi_{\theta_1}(\cdot|s)\left|\right|\pi_{\theta_2}(\cdot|s))\right]$$

## Sample-Based Estimation

This section describes how the objective and constraint
functions can be approximated using Monte Carlo
simulation. We seek to solve the following optimization problem, obtained
by expanding $L_{\theta_{\text{old}}}$:

$$\underset{\theta}{\text{maximize}}\sum_{s}\rho_{\pi_{\theta_{\text{old}}}}(s)\sum_{a}\pi_{\theta}(a|s)A_{\pi_{\theta_{\text{old}}}}(s,a)$$

$$\text{subject to  }\bar{D}_{\text{KL}}^{\rho_{\theta_{\text{old}}}}(\theta_{\text{old}}, \theta)\le\delta.$$

* We first replace $\sum_{s}\rho_{\pi_{\theta_{\text{old}}}}(s)[\dots]$ in the objective by the expectation $\frac{1}{1-\gamma}\mathbb{E}_{s\sim\rho_{\theta_{\text{old}}}}[\dots]$

* Next, we
replace the sum over the actions by an importance sampling
estimator. Using $q$ to denote the sampling distribution, the
contribution of a single $s_{n}$ to the loss function is

    $$\sum_{a}\pi_{\theta}(a|s)A_{\pi_{\theta_{\text{old}}}}(s,a) = \mathbb{E}_{a\sim q}\left[\frac{\pi_{\theta}(a|s_n)}{q(a|s_n)}A_{\theta_{\text{old}}}(s_n,a)\right]$$
    
Our optimization problem is exactly
equivalent to the following one:

$$\underset{\theta}{\text{maximize}}\ \mathbb{E}_{s\sim\rho_{\theta_{\text{old}}},a\sim q}\left[\frac{\pi_{\theta}(a|s_n)}{q(a|s_n)}A_{\theta_{\text{old}}}(s_n,a)\right]$$

$$\text{subject to  }\mathbb{E}_{s\sim\rho_{\theta_{\text{old}}}}\left[D_{\text{KL}}(\pi_{\theta_{\text{old}}}(\cdot|s)\left|\right|\pi_{\theta}(\cdot|s))\right] \le \delta.$$

In the estimation procedure, we collect a sequence of
states by sampling $s_0\sim\rho_0$ and then simulating the policy $\pi_{\theta_{\text{old}}}$ for some number of timesteps to generate a trajectory $s_0,a_0,s_1,a_1,\dots,s_T,a_T$. Hence, $q(a|s)=\pi_{\theta_{\text{old}}}$.